In [7]:
import generator_object, write_run_plot, serial_sim_tools
import numpy as np

from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools]:
    reload(obj)

Continuuing for John sims with rho = 6.9, N=50, T = 592 Kelvin, Z = 11, Inverse brehmmstralung, Non lte

Varying: rho, T, P, internal energy, charge state, meanfree path


In [8]:
def sim(var1, var2, var3):

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = var1, var2
    z = var3
    N0, N1 = 1, 501 #We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9


    gen.materials_atom(index = 1, quantum_n_max = 3, element= "sn")  
    gen.materials_region(nodes = [N0, N1], elec_temp = T_ev, qstart = True) 

    gen.materials_region_material(rho, n_atom, z, z**2) #

    gen.geometry(type = 'slab')
    gen.geometry_nodes(coordinate = "r", scaling_type  = "geom", nodes = [N0, N1], nodes_range = [Rmin, Rmax], drmin = 1.e-6, slope = -1)


    gen.sources_laser(index = 1, laser_wavelength = 1, option_1= 'value', option_2= 'history', id_value = 1, multiplier=1,)
    gen.sources_lasray(entrance_position =  10, entrance_direction_mu = 1, entrance_direction_phi = 0, fractional_power= 1, res_frac=.5)
    
    gen.sources_history(id = 1, value_multiplier = 1e18, time_multiplier = 1., pulse_type='gaussian', p1 = 5*p2, p2 = p2)

    gen.popular_switches(timestep_between_snapshot = 100, 
                        temparture_calc_heating_rates = ['temp calc = time dependant', 'heating rates = electronic'], 
                        raytrace = True, 
                        include_degeneracy = 'no degeneracy',
                        continuum_transfer_evolves_temp = True)

        
    gen.other_switches(resonant_absorption_fraction = 'constant value for each ray from lasray', 
                        subcycle_maximum = 1000,
                        do_kinetics_zone_centered = True, 
                        population_calculation = 'time dependent diffusion', 
                        control_calc_thermal_conduct = 'include thermal conduction')
    
    gen.controls(t_start = 0, t_end = 2.e-8, restart = True)

    gen.parameters(time_between_snapshots = 1e-9, initial_timestep = 1.e-14)
    
    return gen




In [9]:
var = np.array([.5, .75, 1, 1.33, 2.])

Tvals = [2*592./11600]
rhovals = 6.9*var
z_vals = [12]

combi = serial_sim_tools.combinatorics([Tvals, rhovals, z_vals])
#for comb in combi:
#    print(comb)
sims = []



for entry in combi:
    base_name = 'johnrun1'
    [var1,var2,var3] = entry
    var1, var, var3 = round(var1,3), round(var2,3), round(var3,3)
    gen = sim(var1, var2, var3)
    name = f'{base_name}_{var1}_{var2}_{var3}'
    sims.append(name)
    write_run_plot.write(name=name, longprint = False, plot_duplicates=False, object=gen)
    #write_run_plot.run(name=name, longprint = False, plot_duplicates=False, object=gen)
    #write_run_plot.plot_dump(name=name, longprint = False, plot_duplicates=False, object=gen)
#serial_sim_tools.plot_all('sims_multiplot', sims)

# i'm seeing 2 minutes for 2 options, so 5*5*3 = 75 minutes maybe a bit more 


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.102_3.45_12/johnrun1_0.102_3.45_12.gen

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.102_5.175000000000001_12/johnrun1_0.102_5.175000000000001_12.gen

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.102_6.9_12/johnrun1_0.102_6.9_12.gen

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.102_9.177000000000001_12/johnrun1_0.102_9.177000000000001_12.gen

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.102_13.8_12/johnrun1_0.102_13.8_12.gen


 

on mean free paths:
Rosseland mean absorption coefficient, including scattering, at node ir (1/cm)
Planck mean absorption coefficient, not including contributions of photoexcitations, at node ir (1/cm)

on internal energy:
Internal energy of element iz at node ir with no continuum lowering (erg/cm3).


In [10]:
# Looking for temperature, charge state, laser absorption pe unit distance, radiative heat. & other results from theoretical predictions

def sim2(var2):

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = 15, .1
    z = 12
    N0, N1 = 1, 501 #We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9


    gen.materials_atom(index = 1, quantum_n_max = 3, element= "sn")  
    gen.materials_region(nodes = [N0, N1], elec_temp = T_ev, qstart = True) 

    gen.materials_region_material(rho, n_atom, z, z**2) #

    gen.geometry(type = 'slab')
    gen.geometry_nodes(coordinate = "r", scaling_type  = "geom", nodes = [N0, N1], nodes_range = [Rmin, Rmax], drmin = 1.e-6, slope = -1)


    gen.sources_laser(index = 1, laser_wavelength = 1, option_1= 'value', option_2= 'history', id_value = 1, multiplier=1,)
    gen.sources_lasray(entrance_position =  10, entrance_direction_mu = 1, entrance_direction_phi = 0, fractional_power= 1, res_frac=.5)
    
    gen.sources_history(id = 1, value_multiplier = 1e18, time_multiplier = 1., pulse_type='gaussian', p1 = 5*p2, p2 = p2)

    gen.popular_switches(timestep_between_snapshot = 100, 
                        temparture_calc_heating_rates = ['temp calc = time dependant', 'heating rates = electronic'], 
                        raytrace = True, 
                        include_degeneracy = 'no degeneracy',
                        continuum_transfer_evolves_temp = True)

        
    gen.other_switches(resonant_absorption_fraction = 'constant value for each ray from lasray', 
                        subcycle_maximum = 1000,
                        do_kinetics_zone_centered = True, 
                        population_calculation = 'time dependent diffusion', 
                        control_calc_thermal_conduct = 'include thermal conduction')
    
    gen.controls(t_start = 0, t_end = 2.e-8, restart = True)

    gen.parameters(time_between_snapshots = 1e-9, initial_timestep = 1.e-14)
    
    return gen

#write_run_plot.write()


In [11]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)


def sim3():

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = 6.9, .1
    z = 12
    N0, N1 = 1, 501  # We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9

    gen.materials_atom(index=1, quantum_n_max=3, element="sn")
    gen.materials_region(nodes=[N0, N1], elec_temp=T_ev, qstart=True)

    gen.materials_region_material(rho, n_atom, z, z**2)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate="r", scaling_type="geom", nodes=[
                       N0, N1], nodes_range=[Rmin, Rmax], drmin=1.e-6, slope=-1)

    gen.sources_laser(index=1, laser_wavelength=1, option_1='value',
                      option_2='history', id_value=1, multiplier=1,)
    gen.sources_lasray(entrance_position=10, entrance_direction_mu=1,
                       entrance_direction_phi=0, fractional_power=1, res_frac=.5)

    gen.sources_history(id=1, value_multiplier=1e20,
                        time_multiplier=1., pulse_type='gaussian', p1=5*p2, p2=p2)

    gen.popular_switches(temparture_calc_heating_rates=[
                            'temp calc = steady state', 'heating rate uses internal energy rates'],
                         raytrace=True,
                         include_degeneracy='no degeneracy',
                         continuum_transfer_evolves_temp=True)

    gen.other_switches(resonant_absorption_fraction='constant value for each ray from lasray',
                       subcycle_maximum=1000,
                       do_kinetics_zone_centered=True,
                       population_calculation='time dependent diffusion',
                       control_calc_thermal_conduct='include thermal conduction')
    
    #gen.rswitches(Rad_transfer_alg='1-d and do transport using integral formalism')

    gen.controls(t_start=0, t_end=2.e-8, restart=True)

    gen.parameters(time_between_snapshots=1e-9, initial_timestep=1.e-14)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]}) # works in linear
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  # works in linear

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'te':[0,0]}) # works in log
    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]}) # works in log

    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) #doesn't work 
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) #doesn't work 
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'tr':[0,0]})#doesn't work 
    return gen


name = 'johnrun6'
gen = sim3()

#write_run_plot.write(name=name, longprint=False,plot_duplicates=False, object=gen)
#write_run_plot.run(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.plot_dump(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.extra_plot(name = name,  multiplot=False, make_animation=False)



In [12]:
import search
reload(search)

search_for_word = 'isorange'

df = search.Generators().word(input = search_for_word, word_slicing = (0,5), print_path = True)
print(df.to_string())


search_inside_chapter = 'Switches and Parameters'
#print(search.Generators().chapter(input = search_inside_chapter))


/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//nlte-w/nlte-w.gen
                     0
isorange all 2 1     3
isorange all 10 0 3  1


In [13]:
def remove(name):
    import os, shutil
    path_rem = f'/home/brewster/Desktop/cretin_package-master/Personal_experiments/{name}/images'

    if os.path.isdir(path_rem):
        print(f'\nremoving {path_rem}')
        shutil.rmtree(path_rem)

In [14]:
for obj in [generator_object, write_run_plot, serial_sim_tools]:
    reload(obj)

name = 'sn_rad_test'
gen = sim3()
remove(name)

write_run_plot.write(name=name, longprint=False,plot_duplicates=False, object=gen)
write_run_plot.run(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.plot_dump(name=name, longprint = False, plot_duplicates=False, object=gen)
write_run_plot.extra_plot(name = name,  multiplot=False, make_animation=False)


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/sn_rad_test/sn_rad_test.gen
running cretin with sn_rad_test
adding additional plot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/sn_rad_test/images
Created image folder at /home/brewster/Desktop/cretin_package-master/Personal_experiments/sn_rad_test/images


In [15]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin():

    gen = generator_object.User_input()
    
    rho, T_ev = 1., .6

    z = 12. #
    N0, N1 = 1, 501  # We are operating in 1d wih 100 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 2*1.06 micrometer

    gen.materials_atom(index = 1, quantum_n_max = 3, element = "sn")
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen



sim4 = lasfoam_tin()
name = 'Howard_scott32'
write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
#write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
#write_run_plot.run(name = name, object = sim4, longprint=False)
#write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
#write_run_plot.extra_plot(name = name, make_animation=True)


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/Howard_scott32/Howard_scott32.gen


In [16]:
import generator_object, write_run_plot, serial_sim_tools, animate, plt_file, to_generator_string


from importlib import reload
for obj in [generator_object, to_generator_string, write_run_plot, serial_sim_tools, animate, plt_file]:
    reload(obj)

def lasfoam_tin(i):

    gen = generator_object.User_input()
    
    rho, T_ev = round(i*1,3), .6
    print(rho)
    z = 12. #
    N0, N1 = 1, 501  # We are operating in 1d wih 500 nodes
    Rmin, Rmax = 0, 30.e-4 #node min max max = 10^-6m = 30 micron 
    ILASER, TLASER = 1e18, 1.e-9 #corresponds to 10^11 W/cm^2
    wavelength_laser = 1 # corresponds to 2*1.06 micrometer

    gen.materials_atom(index = 1, quantum_n_max = 3, element = "sn")
    gen.materials_region([N0,N1], elec_temp = T_ev)
    gen.materials_region_rho(rho)
    gen.materials_region_element(initial_ion_population = 1.0, index = 1)

    gen.geometry(type='slab')
    gen.geometry_nodes(coordinate = "r", scaling_type = "lin", nodes = [N0, N1], nodes_range = [Rmin, Rmax])

    gen.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen.radiation_angles(n_rays = 3)

    gen.controls(t_start = 0., t_end = 3e-9, restart = True)

    gen.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen.controls_history_tv(time = 0., value = 1)
    gen.controls_history_tv(time = 1.0, value = 1.)
    gen.controls_history_tv(time = 1.0, value = 0.)

    gen.sources_source_laser(laser_wavelength = wavelength_laser, option_1 = 'rate', option_2 = 'history', values = [1,1.], nodes = [N0, round(N1/10)])
    gen.parameters(scattering_multiplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-16, maximum_timestep = 10**-10, time_between_snapshots = 10**(-9))

    gen.popular_switches(continuum_transfer_evolves_temp = True,
                         mesh_treatment = 'staggered centering (node + zones)')
    gen.other_switches(control_calc_thermal_conduct = 'include thermal conduction', 
                       population_calculation = "time dependent diffusion", 
                       temperature_calculation = 'use electronic heating rates for time-dependent')
    
    gen.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)

    gen.add_plot(title='electron number density', xvars={'time':[],'ir':[0,0]}, yvars={'ne':[0,0]})
    gen.add_plot(title='mass density', xvars={'time':[],'ir':[0,0]}, yvars={'zrho':[0,0]})  
    gen.add_plot(title='charge state', xvars={'time':[],'ir':[0,0]}, yvars={'zbar':[0,0]}) 

    gen.add_plot(title='electron temperature', xvars={'time':[],'ir':[0,0]}, yvars={'tev':[0,0]})
    gen.add_plot(title='radiation remperature', xvars={'time':[],'ir':[0,0]}, yvars={'trv':[0,0]}) 

    gen.add_plot(title='net energy gain due to laser absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatl':[0,0]})  
    gen.add_plot(title='net energy gain due to radiation absorption', xvars={'time':[],'ir':[0,0]}, yvars={'heatj':[0,0]}) 
    gen.add_plot(title='Net free electron heating rate due to all processes', xvars={'time':[],'ir':[0,0]}, yvars={'heatt':[0,0]}) 

    return gen


for i in range(2,12):
    sim4 = lasfoam_tin(1.2**i)
    name = f'Howard_scott32_{i}'
    #write_run_plot.write(name = name, object = sim4, longprint=False, plot_duplicates=False)
    #write_run_plot.all(name = name, object = sim4, longprint=False, plot_duplicates=False)
    #write_run_plot.run(name = name, object = sim4, longprint=False)
    #write_run_plot.plot_dump(name = name, object = sim4, longprint=False)
    #write_run_plot.extra_plot(name = name, make_animation=True)

1.44
1.728
2.074
2.488
2.986
3.583
4.3
5.16
6.192
7.43


atoms hydrogenic sn
  modeltype dca

switch 155 10
param 18 1.0
param 19 1.0

In [ ]:
import search
reload(search)

search_for_word = 'yvar emis_sp'

df = search.Generators().word(input = search_for_word, word_slicing = (0,5), print_path = True)
print(df.to_string())


search_inside_chapter = 'Switches and Parameters'
#print(search.Generators().chapter(input = search_inside_chapter))

/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//thd/thd.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//spec1d/spec1d.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//magslab/magslab.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//nlte-au/nlte-au.gen
/home/brewster/Desktop/cretin_package-master/Premade_cretin_tests//edist/edist.gen
                         0
yvar emis_sp+ 1 n1 0     4
yvar emis_s